### Building a Tree Code VI: A Plummer Model

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
import numpy as np
from timerclass import Timer

from plummer import PlummerModel
from treeclasses import ParticleSet, BHTree

#import numpy.random as rng
#rng.seed(32410)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plt.rcParams['figure.dpi'] = 150

Test of n-body machinery using a Plummer sphere. The density is given by
$$ \rho(r) = \frac{3GM}{4\pi b^3} (r^2+b^2)^{-5/2} $$
The dynamical time (the crossing time for an orbit) is
$$ t_d = \sqrt{\frac{3\pi}{16G\rho_0}}$$

The timescale for two-body effects to become important if we use a softening length of $\epsilon$ is
$$ t_r = \frac{N}{8\ln(R/\epsilon)} t_c $$

In [ ]:
def rho(r, GM, b):
    return 3*GM/(4*np.pi*eps) * (r**2 + b**2)**(-2.5)

def tDynamical(Grho0):
    return np.sqrt( 3*np.pi/(16*Grho0) )

def tRelax(rho0, R, eps, N):
    tc = tDynamical(rho0)
    return N/(8*np.log(R/eps))*tc

M = 1
b = 1

N = 2**16
eps = 1e-2
R = 1

rho0 = rho(0, M, b)
print(f"rho0 = {rho0}")
print(f"t_d = {tDynamical(rho0)}")
print(f"t_r = {tRelax(rho0, R, eps, N)}")

For these parameters, we would not expect two-body effects to have much effect if we run for less than,
say, a dimensionless time $t\lesssim100$. We won't get much beyond $t=1$ in this exercise.

To run an NBody calculation, we need a time-integration framework. We will start with a container for the state of the system, the time, position, velocities, *etc.* It seemed logical to include routines here for the kick and drift operators, using another class (referenced here as hamilton) to solve for the right-hand sides, just as we did in our previous ODE solvers.

The particles will be in a ParticleSet, here given as the argument ps:

In [ ]:
class State:
    def __init__(self, ps, hamilton ):
        self.time = 0.0
        self.step = 0
        self.ps = ps
        
    def kick(self, h, hamilton, recalculate):
        # Update velocity 
        self.ps.v += h * hamilton.momentumEquation(self.ps, recalculate)
        return self
    
    def drift(self, h, hamilton, recalculate):
        # Update positions
        self.ps.r += h * hamilton.positionEquation(self.ps, recalculate)
        return self

The Hamiltonian then solves for the acceleration given the state, implementing the functions used by State to implement kick and drift. Solving for the acceleration is done using the Barnes-Hut tree walk from treeclasses.py, the wrapper of the C++ inplementation of Barnes-Hut. (If you get an error about opening a shared object file when executing below, you will need to type "make lib" in the directory where treeclasses.py is located.)

One can set the ```verbose``` flag to true to get information on the tree build, ```check``` to true to check that the tree build was correct, and  ```getStats``` to print information on the interaction lists.

In [ ]:
class Hamiltonian:
    def __init__(self, theta, epsSmooth, maxLeafSize, maxSrcLen):
        self.theta = theta
        self.epsSmooth = epsSmooth
        self.maxLeafSize = maxLeafSize
        self.maxSrcLen = maxSrcLen
        
        self.verbose = False
        self.check = False
        self.getStats = False
        
    def getAcceleration(self, ps):
        
        ps.computeBoundingBox()
        center, halfWidth = ps.getBoundingBox()
        
        BH = BHTree(ps, self.maxLeafSize, self.epsSmooth, self.maxSrcLen)

        BH.makeTree(self.verbose, self.check, self.getStats)

        BH.BHsubsets(theta, ps.N)     # use leaves as sink sets
        #BH.accAll(theta)             # use single particles as sinks
        
        BH.putParticlesInOriginalOrder()  # so we know which particles are which galaxy
        
        BH.free()                         # important to free memory on the C side...
        
    def positionEquation(self, ps, newvel):
        return ps.v
    
    def momentumEquation(self, ps, newacc):
        if newacc: self.getAcceleration(ps)
        return ps.a

With these two classes, it is simple to implement leapfrog as the product of kick and drift operators:

In [ ]:
def KDK(dt, hamilton, s):
    s = s.kick(dt/2, hamilton, False).drift(dt, hamilton, False).kick(dt/2, hamilton, True)
    s.time += dt
    s.step += 1
    return s

Here is a class to compute the total kinetic and potential energies. We'll use this to keep track of conservation of energy as the simulation proceeds.

In [ ]:
class Conservation:
    def __init__(self, ps):
        self.ps = ps
        self.K0, self.P0, self.E0 = self.getEnergy()

    def getEnergy(self):
        K = 0.5*np.sum(self.ps.mass * np.sum(self.ps.v**2, axis=1))
        P = 0.5*np.sum(self.ps.mass * self.ps.pot)
        return K, P, K+P

With these, we are now ready to create a ParticleSet and fill it with a Plummer model. We'll choose 256K particles to begin
with:

In [ ]:
N = 2**18
PS = ParticleSet()
PS.reserve(N)

r, v, m = PlummerModel(N)
PS.r[:,:] = r
PS.v[:,:] = v
PS.mass[:] = m

We next set up the problem, using a softening length which is near optimal for this system:

In [ ]:
PS.computeBoundingBox()
centre, halfWidth = PS.getBoundingBox()
maxLeafSize = 32
epsSmooth = 0.98*PS.N**(-0.26) 
maxSources = N
theta = 0.75

H = Hamiltonian(theta, epsSmooth, maxLeafSize, maxSources)
S = State(PS, H)

# take a step of zero time just to get the initial potential energy of the system
S = KDK(0.0, H, S)
C = Conservation(PS)
print(f"kinetic energy: {C.K0:.4e}, potential energy: {C.P0:.4e}, total energy: {C.E0:.4e}")

Note that he virial theorem is nearly satisfied by our Monte Carlo realization of a Plummer model.

Set up a plot to be animated. The particle accelerations and potentials for the initial conditions are
plotted in red, while those computed at the current timestep will be plotted in blue.

To make the plotting faster, we plot only every ```sample```-th point:

In [ ]:
sample = 10

fig, ax = plt.subplots()
axp = ax.twinx()
ax.loglog(np.linalg.norm(S.ps.r[::sample,:], axis=1), np.linalg.norm(S.ps.a[::sample,:], axis=1),'r,')
axp.loglog(np.linalg.norm(S.ps.r[::sample,:], axis=1), -S.ps.pot[::sample],'r,')

accplot, = ax.semilogy([],[],',')
potplot, = axp.semilogy([],[],'g,')
steptxt = ax.text(0.2,1.1,f"step: {0:4d}   time: {0:8.2e}", transform=ax.transAxes)
fig.tight_layout()

Here is a function to update the plot after each timestep

In [ ]:
def updatePlot(S):
    K, P, E = C.getEnergy()
    print(f"{S.step:4d} {S.time:.4e}  {K:.4e} {P:.4e} {E:.4e} {(E-C.E0)/C.E0:.2e}")
    accplot.set_data(np.linalg.norm(S.ps.r[::sample,:], axis=1), np.linalg.norm(S.ps.a[::sample,:], axis=1))
    potplot.set_data(np.linalg.norm(S.ps.r[::sample,:], axis=1), -S.ps.pot[::sample])

    steptxt.set_text(f"step: {S.step:4d}   time: {S.time:8.2e}   E cons: {E-C.E0:.2e}  {(E-C.E0)/C.E0:.2e}")
    plt.gcf().canvas.draw_idle()
    plt.gcf().canvas.start_event_loop(0.001)

Run the simulation! We expect this system to be nearly in equilibrium, so we don't expect much evolution.

In [ ]:
print("step    time        K           P           E           dE")

dt = 0.01
while(S.time<100):
    
    S = KDK(dt, H, S) # take a KDK step

    if S.step%10==0:
        updatePlot(S)
        
K, P, E = C.getEnergy(PS)
updatePlot(S)